In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import numpy as np
import librosa
import zipfile
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential#sequential is a api
from tensorflow.keras.layers import Dense, Flatten# dense is fully connected layer connects every neuron in layer too previous layer, flatten convert multidimensional in one dimensional
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint#earlystopping stops the training process if model performance stops improving some factors passed monitor-jiske basis pe decide ,patience-no of epochs jab tak aisa ho learningratescheduler changes is a function that we can paas to change the learningrate after a set of epochs Modelcheckpoint save the model weights at a particulat time it takes input filepath wherew e need to save monitor- the function jiske basis par decide
from sklearn.metrics import classification_report#basically gives the report of our model it gives precision - true positive/true positive+false positive(precision 0.8 for class1 mean jitne bhi apan ne class1 predict ki h usme se 80% vo class1 hi h )   recall-tp/tp+fn(0.8 means 100 actual instances h class1 ke and apan ne 80% sahi predict kiya h  ) f1score-weighted average of precision and recall and support- means x instances class1 ki h
from sklearn.preprocessing import LabelEncoder, OneHotEncoder#labelencoder numbers de deta  h 1 -x while onehotencoder make a 0,1 ka matrix
from tensorflow.keras.regularizers import l2 #l2 regularization

def extract_features(filename):#audio is 1d series
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast')#resamplin audio -changing the sample rate of audio signal to make sure files are comaptible with intended playback or processing system

    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)#mfccs-short term power spectrum of sound in a way that models human being -effective for distinguishing phonetic and speech signal, chroma-represent 12 different pitch classes-related to music captures musical chords,mel spectogram-frequencies are converted to mel scale-visualize frequency content overtime how wnergy distribution changes
    mfccs = np.mean(mfccs.T, axis=0)#mfcc is 2d array with shape(n_mfcc,T) where T is no of frames in audio after mean shape becomes (n_mfcc,)

    chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
    chroma = np.mean(chroma.T, axis=0)#(12,T) after mean (12,)

    mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
    mel = np.mean(mel.T, axis=0)#(128,) there are 128 mel bands by default

    spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
    spectral_contrast = np.mean(spectral_contrast.T,axis=0)


    return np.hstack([mfccs, chroma, mel,spectral_contrast])# reduce the dimension by 1  hence resulting wil have shape(180,)

def load_data(base_directory):
    sounds = []
    labels = []
    label_encoder = LabelEncoder()
#in mnist or any other dataset pixels value lie from 0 to 256 so we divide by 255 to scale from 0 to 1
    for i in range(1, 25):
        actor_dir = actor_dir = f"/content/Actor_{i:02d}"
        if os.path.isdir(actor_dir):
            for root, dirs, files in os.walk(actor_dir):
                for file in files:
                    if file.endswith(".wav"):
                        sound_path = os.path.join(root, file)
                        label = os.path.basename(root)
                        labels.append(label)
                        features = extract_features(sound_path)
                        sounds.append(features)

    sounds = np.array(sounds)
    labels = label_encoder.fit_transform(labels)
    labels = OneHotEncoder().fit_transform(labels.reshape(-1, 1)).toarray()
    return sounds, labels

# Define paths
zip_file_path = "/content/drive/MyDrive/Audio_Speech_Actors_01-24.zip"
extraction_path = "/content/"

# Extract zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# Base directory where actors' folders are extracted
base_directory = os.path.join(extraction_path, "Audio_Speech_Actors_01-24")

# Load data
sounds, labels = load_data(base_directory)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sounds, labels, test_size=0.1, random_state=42)

# Define the model
model = Sequential([
    Dense(250, activation='relu', input_shape=(sounds.shape[1],), kernel_regularizer=l2(0.01)),
    Dense(150, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(75, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(labels.shape[1], activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#Uses moving averages of the gradient (first moment) and the squared gradient (second moment) to adapt the learning rate for each parameter first moment estimate is mean and second is uncertain variance, adam optimizer-

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
learning_rate_scheduler = LearningRateScheduler(lambda epoch: 1e-3 * 10 ** (-epoch / 20))
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

# Train the model
model.fit(X_train, y_train,
          batch_size=32,#number of training samples used in one iteration to update the model's parameters (weights and biases), to complete one epoch model will need number of samples/40 iterations,arger batch sizes take longer per iteration but may complete epochs faster as fewer updates are needed and vice versa
          epochs=80,
          verbose=1,#verbose refers tolevel of detail printed v=0 mean no output shown v=1 mean display proggress for each epoch,
          validation_split=0.1,
          callbacks=[early_stopping, learning_rate_scheduler, model_checkpoint])

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = np.sum(y_pred_classes == y_test_classes) / len(y_test_classes)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

# Print classification report
print(classification_report(y_test_classes, y_pred_classes))

Epoch 1/80
37/37 [==============================] - 1s 12ms/step - loss: 12.9820 - accuracy: 0.0523 - val_loss: 4.9328 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 2/80
29/37 [======================>.......] - ETA: 0s - loss: 4.3579 - accuracy: 0.1067

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


37/37 [==============================] - 0s 5ms/step - loss: 4.2586 - accuracy: 0.1132 - val_loss: 3.8480 - val_accuracy: 0.1154 - lr: 8.9125e-04
Epoch 3/80
37/37 [==============================] - 0s 6ms/step - loss: 3.5104 - accuracy: 0.2273 - val_loss: 3.4574 - val_accuracy: 0.1846 - lr: 7.9433e-04
Epoch 4/80
37/37 [==============================] - 0s 6ms/step - loss: 3.1443 - accuracy: 0.3027 - val_loss: 3.1595 - val_accuracy: 0.2462 - lr: 7.0795e-04
Epoch 5/80
37/37 [==============================] - 0s 6ms/step - loss: 2.8987 - accuracy: 0.3482 - val_loss: 2.9417 - val_accuracy: 0.3462 - lr: 6.3096e-04
Epoch 6/80
37/37 [==============================] - 0s 6ms/step - loss: 2.6579 - accuracy: 0.4057 - val_loss: 2.7277 - val_accuracy: 0.3846 - lr: 5.6234e-04
Epoch 7/80
37/37 [==============================] - 0s 6ms/step - loss: 2.5774 - accuracy: 0.3851 - val_loss: 2.5767 - val_accuracy: 0.4231 - lr: 5.0119e-04
Epoch 8/80
37/37 [==============================] - 0s 6ms/step - los